In [1]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import cursor
import time

In [2]:
# Key API Alpha Vantage
api_key = "9S0PXMUNZHCANG2F"

In [3]:
# List of Stocks
symbols = ["AAPL", "MSFT", "GOOGL", "AMZN"]

In [4]:
# Connect to server MySQL
conn = mysql.connector.connect(
    host="localhost", 
    user="root",  ### Change to specific user
    password="",  ### Use your password
    database="stocksdata"  ### Change to your DB
)

In [5]:
# Declare a dictionary to store the DataFrames for each symbol
dati_per_simbolo = {}

In [6]:
while True:

 # Download data and enter it into the database for each symbol
 for symbol in symbols:
    # Download data from alphavantage
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={api_key}"
    response = requests.get(url)
    data = response.json()

    # Extract the data from the JSON dictionary into a pandas DataFrame
    df = pd.DataFrame.from_dict(data['Time Series (5min)'], orient='index')
    df.index = pd.to_datetime(df.index)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    
    # Sort the DataFrame for the 'Data' column in ascending order
    df = df.sort_index()
    
    # Store the DataFrame in the dictionary
    dati_per_simbolo[symbol] = df
    
   
    
    

    # Create a separate table for the symbol in the database
    cursor = conn.cursor()
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {symbol} (
        id INT AUTO_INCREMENT PRIMARY KEY,
        Date DATETIME UNIQUE ,
        Open FLOAT,
        High FLOAT,
        Low FLOAT,
        Close FLOAT,
        Volume INT
    )
    """
    cursor.execute(create_table_query)

    # Enter the data into the database
    for index, row in df.iterrows():
        cursor.execute(f"INSERT INTO {symbol} (Date, Open, High, Low, Close, Volume) VALUES (%s, %s, %s, %s, %s, %s)",
                       (index, row['Open'], row['High'], row['Low'], row['Close'], row['Volume']))
     
     
    conn.commit()
    cursor.close()

    print("Done!")
    # Wait 5 seconds before running the script again
    
    time.sleep(5)  # 5 seconds

Done!
Done!
Done!
Done!


KeyboardInterrupt: 